In [1]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
import pandas as pd
import json
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup
from shapely.geometry import Polygon, LineString
# import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
# from scipy import stats
from pathlib import Path
# import itertools
import os
# import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path

# geo libraries
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt
from geopy.distance import geodesic
from geopandas import GeoDataFrame
import shapely.ops
import shapely.geometry
from shapely.geometry import Polygon, LineString
cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
# import seaborn as sns
# import matplotlib as mpl 
# %matplotlib inline 
# import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
conn = sqlite3.connect('file_both.db')
vessel_finder = pd.read_sql_query("SELECT * FROM VesselLocation", conn)
#vessel_finder.sort_values(by='DWT', ascending=False, inplace=True)
vessel_finder.query("(`lat` >= -90 and `lat` <= 90) or (`lon` >= -180 and `lon` <= 180)", inplace = True)
vessel_finder.dropna(subset=['lat', 'lon'], inplace=True)
geometry = [Point(xy) for xy in zip(vessel_finder.lon, vessel_finder.lat)]
# vessel_finder = vessel_finder.drop(['lon', 'lat', 'mmsi', 'no_pc'], axis=1)
vessel_finder = GeoDataFrame(vessel_finder, crs="EPSG:4326", geometry=geometry)
#vessel_finder.head(2)     

In [3]:
marine_traffic = pd.read_sql_query("SELECT * FROM MarineTrafficVesselLocation", conn)
marine_traffic.query("(`lat` >= -90 and `lat` <= 90) or (`lon` >= -180 and `lon` <= 180)", inplace = True)
marine_traffic.dropna(subset=['lat', 'lon'], inplace=True)
geometry = [Point(xy) for xy in zip(marine_traffic.lon, marine_traffic.lat)]
marine_traffic = GeoDataFrame(marine_traffic, crs="EPSG:4326", geometry=geometry)
# marine_traffic = marine_traffic.drop(['lon', 'lat'], axis=1)
#vessel_finder = vessel_finder[~(vessel_finder['date'] == '2022-12-20')]
#marine_traffic.tail(2)

In [4]:
myshiptracking = pd.read_sql_query("SELECT * FROM MyShipTrackingVesselLocation", conn)
myshiptracking.query("(`lat` >= -90 and `lat` <= 90) or (`lon` >= -180 and `lon` <= 180)", inplace = True)
myshiptracking.dropna(subset=['lat', 'lon'], inplace=True)
geometry = [Point(xy) for xy in zip(myshiptracking.lon, myshiptracking.lat)]
myshiptracking = GeoDataFrame(myshiptracking, crs="EPSG:4326", geometry=geometry)
# myshiptracking.tail(2)

In [5]:
# duplicates = vessels[vessels.duplicated(keep=False)]

In [6]:
vessels = vessel_finder.copy()
vessels = vessels.merge(marine_traffic[['imo', 'received', 'scrape_date', 'geometry', 'lat', 'lon']],
                        on=['imo', 'received', 'scrape_date', 'geometry', 'lat', 'lon'], how='outer')

vessels = vessels.merge(myshiptracking[['imo', 'received', 'scrape_date', 'geometry', 'lat', 'lon']],
                        on=['imo', 'received', 'scrape_date', 'geometry', 'lat', 'lon'], how='outer')

vessels = vessels.drop_duplicates(subset=["received", "geometry"], keep='first')


#del [marine_traffic, myshiptracking]

# converting the string to datetime format
vessels['received'] = pd.to_datetime(vessels['received'], format='%Y-%m-%d %H:%M')
vessels['scrape_date'] = pd.to_datetime(vessels['scrape_date'], format='%Y-%m-%d %H:%M')

vessels.sort_values(by=['received'], ascending=True, na_position='first', inplace=True)


# marine_traffic = marine_traffic.drop(['lon', 'lat'], axis=1)
vessels.reset_index(drop=True, inplace=True)
print(f'Number of ALL vessel positions: {vessels.shape[0]}')
vessels = vessels[(vessels['received'] > "2022-12-30")]
# vessels.query("`received` >", inplace = True)
print(f'Number of vessels with recent positions: {vessels.shape[0]}')
# vessels.head(2)

Number of ALL vessel positions: 5280
Number of vessels with recent positions: 4632


In [7]:
vessels[vessels['imo'] == 9236614].head(20)

,imo,lat,lon,fromPort,toPort,received,eta,atd,ship_cog,ship_sog,ship_type,no_pc,mmsi,scrape_date,geometry
3129,9236614,31.37441,121.6948,NaN,NaN,2023-01-04 15:52:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-04 17:12:00,POINT (121.69480 31.37441)
3682,9236614,31.37446,121.6951,NaN,NaN,2023-01-05 10:27:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-05 13:41:00,POINT (121.69510 31.37446)
4354,9236614,31.37454,121.6951,NaN,NaN,2023-01-06 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-06 10:05:00,POINT (121.69510 31.37454)
5024,9236614,31.37445,121.6949,NaN,NaN,2023-01-06 17:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-06 18:59:00,POINT (121.69490 31.37445)


In [8]:
df = vessels.copy()
df = df.groupby(['imo']).agg({'imo':['last'], 'lat':['last'], 'lon':['last'], 'received':['last']})
df.columns = df.columns.droplevel(0)
df.columns = ['imo', 'lat', 'lon', 'last']
df.reset_index(drop=True, inplace=True)
print(f'length: {df.shape[0]}')
df.head()

length: 577


,imo,lat,lon,last
0,7382732,3.01721,9.835647,2023-01-02 04:13:00
1,7390143,59.35907,5.300120,2023-01-06 18:06:00
2,8110203,2.33091,102.007300,2023-01-06 17:56:00
3,8608872,-5.71445,125.469100,2023-01-06 06:02:00
4,8608884,34.24350,139.922900,2023-01-06 17:50:00


In [28]:
def get_port_distance(lat, lon, max_distance, df):
    """
    latitude and longitude of the port and maxiaml distance of the vessel to the port
    """
    dist_list = []
    for index, row in df.iterrows():
        try:
            origin = (row['lat'], row['lon'])  # (latitude, longitude) don't confuse
            dist = (lat, lon)
            distance = round(geodesic(origin, dist).kilometers)
            # print(row['imo'], distance)
            if distance < max_distance:
                dist_list.append(row['imo'])
        except Exception as e:
            # print(e)
            pass
    return dist_list

def radius_around_port(lat, lon, radius_in_meter):
    radius = gpd.GeoDataFrame({'ship': f'test', 'geometry': [Point(lon, lat)]}, crs='EPSG:4326')
    radius = radius.to_crs(epsg=7855).buffer(radius_in_meter).to_crs(epsg=4326)
    boundary_geojson = gpd.GeoSeries(radius)
    region = boundary_geojson.geometry.unary_union
    return region


def get_line(poly):
    try:
        ls = shapely.geometry.LineString(poly.boundary.coords)
        lats, lons = ls.coords.xy
        return pd.Series([lats, lons])
    except Exception as e:
        return pd.Series([np.nan, np.nan])

t0 = time.time()
ports = pd.read_csv('./data/GIE_LNGs_V1.csv', index_col=False)
ports['bubble'] = ports.apply(lambda x: radius_around_port(x['lat'], x['lon'], radius_in_meter=5000), axis=1)
print(f"Processing took: {round(time.time()-t0)} s")
t0 = time.time()
ports[['lats','lons']] = ports.apply(lambda x: get_line(x['bubble']), axis=1)
print(f"Processing took: {round(time.time()-t0)} s")
t0 = time.time()
# ports['vessels_close'] = ports.apply(lambda x: get_port_distance(x['lat'], x['lon'], 5, df), axis=1)
print(f"Processing took: {round(time.time()-t0)} s")
# ports.tail(5)

IllegalArgumentException: Operation not supported by GeometryCollection

IllegalArgumentException: Operation not supported by GeometryCollection

IllegalArgumentException: Operation not supported by GeometryCollection

IllegalArgumentException: Operation not supported by GeometryCollection



Processing took: 6 s
Processing took: 0 s
Processing took: 0 s


### 2. Charts
#### 2.1. Latest vessel position 

In [ ]:
# %load scripts/draw_map.py

def add_ship(fig, df, col="#76b5c5", size=12):
    try:
        print(df.head())
        #template=f"<br>Name: {df['imo']}</b><extra></extra>"
        fig.add_scattermapbox(
            lat=df['lat'].tolist(), lon=df['lon'].tolist(), mode='markers',
            #hovertemplate=template,
            marker_size=size, marker_color=col,
            line=dict(width=6),opacity= 0.4, hoverinfo='none',
            showlegend=False,)
    
    except Exception as e:
        print(e)

        
def add_radius(fig, df):
    try:
        radius_in_meter = 10000
        fig.add_trace(go.Scattermapbox(
                mode="lines",
                lon=list(df['lats']),
                lat=list(df['lons']),
                name=f"{radius_in_meter} m radius",
                hoverinfo='skip',
                showlegend=False,
                marker={'size': 15, 'color': 'red', 'opacity': 0.6}))
    except Exception as e:
        pass

    
def add_ports(fig, df, col="#bbb5c5", symbol="harbor", size=12):
    try:
        if symbol == "harbor":
            fig.add_scattermapbox(
            name='Home', lat=df['lat'].tolist(), lon=df['lon'].tolist(),
            mode='markers+text',
            marker=dict(size=size, color=col),
            text=df['Name'].tolist(),
            textposition="bottom center",
            hoverinfo='text',
            textfont=dict(color='black', size=10),
            showlegend=False,)
                
        else:
            #template=f"<br>Name: {df['imo']}</b><extra></extra>"
            fig.add_scattermapbox(
                name='Home',
                lat=df['lat'].tolist(), lon=df['lon'].tolist(),
                mode='markers',
                #hovertemplate=template,
                #marker_size=size, 
                #marker_color=col,
                marker=dict(size=size, symbol=symbol, color=col),
                #line=dict(width=6),
                #if symbol == "harbor":
                #    text=df['Name'].tolist(),
               #    textposition="bottom center",
                #    hoverinfo='text',
               #     textfont=dict(color='black', size=10),

                showlegend=False,)

    except Exception as e:
        print(e)

def world_map(vessels, ports):
    fig = go.Figure()
    
    # add traces
    add_ports(fig, ports, col="#76b5c5", symbol="harbor", size=16)
    add_ports(fig, vessels, col="#be4d25", symbol="marker", size=10)
    
    for index, row in ports.iterrows():
        add_radius(fig, row)

        
    fig.update_layout(mapbox_style="light", mapbox_accesstoken=
                      "pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                      mapbox_zoom=3, mapbox_center={"lat": 47.809490, "lon": 13.055010})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(title_text = 'Last position of the vessels (todo: check if correct)')
    fig.show()


world_map(df, ports)

#### 2.2. Vessel Traces 

In [27]:
dft = vessels.copy()
dft = dft.groupby('imo').agg({'lat':lambda x: list(x), 'lon':lambda x: list(x)})
dft.tail()

def add_trace(fig, df, col="#76b5c5", size=12):
    try:
       # template=f"<br>Name: {df['imo']}</b><extra></extra>"
        fig.add_scattermapbox(
            lat=df['lat'], lon=df['lon'], mode="markers+text+lines", line=dict(width=1),
            #hovertemplate=template,
        showlegend=False)
    except Exception as e:
        print(e)

def add_ports(fig, df, col="#bbb5c5", symbol="harbor", size=12):
    try:
        if symbol == "harbor":
            fig.add_scattermapbox(
            name='Home', lat=df['lat'].tolist(), lon=df['lon'].tolist(),
            mode='markers+text',
            marker=dict(size=size, color=col),
            text=df['Name'].tolist(),
            textposition="bottom center",
            hoverinfo='text',
            textfont=dict(color='black', size=10),
            showlegend=False,)
        else:
            #template=f"<br>Name: {df['imo']}</b><extra></extra>"
            fig.add_scattermapbox(
                name='Home',
                lat=df['lat'].tolist(), lon=df['lon'].tolist(),
                mode='markers',
                marker=dict(size=size, symbol=symbol, color=col),
                showlegend=False,)
    except Exception as e:
        print(e)


def world_map_V2(vessel, ports):
    fig = go.Figure()
    add_ports(fig, ports, col="#76b5c5", symbol="harbor", size=16)
    for index, row in vessel.iterrows():
        add_trace(fig, row)

    fig.update_layout(mapbox_style="light", mapbox_accesstoken=
                      "pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                      mapbox_zoom=3, mapbox_center={"lat": 47.809490, "lon": 13.055010})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(title_text = 'Global LNG ports and LNG carrier fleets')
    fig.show()

def test():
    test = dft.loc[dft.index[df['imo'] == 9236614]]
    world_map_V2(test, ports)
    

world_map_V2(dft, ports)

In [22]:
dft.head()
dft.loc[dft.index[df['imo'] == 7390143]]

,lat,lon
imo,,
7390143,"[59.35905, 59.35906, 59.3591, 59.35907, 59.359...","[5.30004, 5.300122, 5.3001, 5.300048, 5.30002,..."


#### 2.3. Vessel Traces

In [14]:
t = vessels[(vessels['imo'] == 7390143) | (vessels['imo'] == 9236614) ]
fig = px.line_mapbox(vessels, lat="lat", lon="lon", color="imo", zoom=3, height=300)
# add_ports(fig, ports, col="#76b5c5", symbol="harbor", size=16)
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox_style="light", mapbox_accesstoken=
                  "pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                  mapbox_zoom=3, mapbox_center={"lat": 47.809490, "lon": 13.055010})
fig.show()

In [29]:
dft2 = vessels.copy()

dft3= dft2.groupby('imo').agg({'lat':lambda x: list(x), 'lon':lambda x: list(x), 'geometry':lambda x: list(x), 'received':lambda x: list(x)})

def generate_linestring(geometry):
    #LineString([Point(0, 0), Point(1, 1)]).wkt
    try:
        # works also with comprehensive list
        return LineString(geometry).wkt
    except Exception as e:
        return -1

dft3['line'] = dft3.apply(lambda x: generate_linestring(x['geometry']), axis=1)

dft3.reset_index(inplace=True)
dft3 = dft3.rename(columns = {'index':'imo'})
dft3.tail()

lats = []
lons = []
names = []
for index, row in dft3.iterrows():
    for i in row['lat']:
        lats = np.append(lats, i)
        
    for i in row['lon']:
        lons = np.append(lons, i)
        
    for i in range(len(row['lat'])):
        names = np.append(names, row['imo'])
    
    lats = np.append(lats, None)
    lons = np.append(lons, None) 
    names = np.append(names, None) 
fig = px.line_geo(lat=lats, lon=lons, hover_name=names)
fig.show()

In [30]:
dft = vessels.copy()
dft= dft.groupby('imo').agg({'lat':lambda x: list(x), 'lon':lambda x: list(x), 'geometry':lambda x: list(x), 'received':lambda x: list(x)})

def generate_linestring(geometry):
    #LineString([Point(0, 0), Point(1, 1)]).wkt
    try:
        # works also with comprehensive list
        return LineString(geometry).wkt
    except Exception as e:
        pass

dft['line'] = dft.apply(lambda x: generate_linestring(x['geometry']), axis=1)

dft.reset_index(inplace=True)
dft = dft.rename(columns = {'index':'imo'})

dft.tail()

,imo,lat,lon,geometry,received,line
572,9905978,"[3.26314, 16.25583, 24.19443, 24.74677]","[113.02988, 119.5473, 122.2647, 122.5409]","[POINT (113.02988 3.26314), POINT (119.5473 16...","[2023-01-02 12:10:00, 2023-01-04 23:27:00, 202...","LINESTRING (113.02988 3.26314, 119.5473 16.255..."
573,9905980,"[22.3739, 15.41367, 3.263978, 3.26751]","[119.83942, 119.0665, 113.0592, 113.00424]","[POINT (119.83942 22.3739), POINT (119.0665 15...","[2022-12-30 22:21:00, 2023-01-01 02:54:00, 202...","LINESTRING (119.83942 22.3739, 119.0665 15.413..."
574,9915894,"[15.74017, 22.58925, 22.16404, 21.97326, 21.97...","[118.2148, 114.4298, 114.59011, 114.60832, 114...","[POINT (118.2148 15.74017), POINT (114.4298 22...","[2023-01-01 19:34:00, 2023-01-03 03:34:00, 202...","LINESTRING (118.2148 15.74017, 114.4298 22.589..."
575,9919890,"[51.3708, 51.28828, 51.04898, 51.04899, 51.048...","[2.5237, 2.409855, 2.36002, 2.360023, 2.36001,...","[POINT (2.5237 51.3708), POINT (2.409855 51.28...","[2023-01-01 21:40:00, 2023-01-01 22:09:00, 202...","LINESTRING (2.5237 51.3708, 2.409855 51.28828,..."
576,9922976,"[1.17637, 1.176228, 1.17609, 1.176168, 1.17614...","[103.77276, 103.7729, 103.77294, 103.7729, 103...","[POINT (103.77276 1.17637), POINT (103.7729 1....","[2023-01-01 21:36:00, 2023-01-01 22:06:00, 202...","LINESTRING (103.77276 1.17637, 103.7729 1.1762..."


In [ ]:
def check_if_coord_in_poly(region, lon, lat):
    """
    Check if a coordinate (lat,long) is within a given polygon
    Used to check for an address in vienna geometry
    source: https://stackoverflow.com/questions/48097742/geopandas-point-in-polygon
    Return: True if Point within Polygon
    
    source: https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python
    
    """
    """
    _pnts = [Point(lon, lat)]
    poly = gpd.GeoSeries({'within': region})
    pnts = gpd.GeoDataFrame(geometry=_pnts, index=['Point to check'], crs=get_local_crs(lon, lat))
    pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in poly.items()})
    return pnts['within'].item()
    """
    point = Point(lon, lat)
    # print(region.contains(point))
    # return (point.within(region))
    #if region.contains(point):
    #    print(region.contains(point))
    return region.contains(point)
    
#for index, row in ports.iterrows():
#    vessel['close_to_harbor'] = vessel.apply(lambda x: check_if_coord_in_poly(row['bubble'], x['lon'], x['lat']), axis=1)
# vessel.head(3)